# Crack Detection In Buildings

Crack detection has vital importance for structural health monitoring and inspection. We would like to train a network to detect Cracks, we will denote the images that contain cracks as positive and images with no cracks as negative.

To train the network, we will use a pre-trained model to classify between the positive and negetive samples. The particular **pre-trained model** will be **resnet18**.

## Download the data

Download the dataset and unzip the files in the data directory. (This may take some time)

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Positive_tensors.zip 

In [ ]:
!unzip -q Positive_tensors.zip 

In [ ]:
! wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Negative_tensors.zip
!unzip -q Negative_tensors.zip

We will install torchvision:

In [ ]:
!pip install torchvision